In [170]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [171]:
meta_input = "../data/input_meta.csv"
df=pd.read_csv(meta_input,low_memory=False)
search_zip_code = df["zip"][0]
search_job_title = df["job_title"][0]

search_zip_code, search_job_title

(95050, 'Data Scientist')

In [172]:
glassdoor_url="https://www.glassdoor.com/index.htm"

In [173]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser1 = Browser('chrome', **executable_path, headless=False)

In [174]:
browser1.visit(glassdoor_url)
browser1.fill('sc.keyword',search_job_title)
browser1.find_by_css('.loc').fill(str(search_zip_code))
browser1.find_by_css('.gd-btn-mkt').first.click()


In [175]:
#get the source code of the browser
html1 = browser1.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html1, 'html.parser')

In [176]:
# Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [177]:
glass_door_data={}
title=[]
company=[]
city=[]
state=[]
url=[]
description=[]

In [178]:
results = soup.find_all("li", class_="jl")
for i in range(len(results)):
    link = results[i].find('div', class_="flexbox").find('a', class_="jobLink")["href"]
    title_text = results[i].find('div', class_="flexbox").find('a', class_="jobLink").text 
    
    title.append(title_text)
    url.append(f'https://www.glassdoor.com{link}')
    
    company.append(results[i].find('div', class_="flexbox empLoc").find('div').text.split(' – ')[0].lstrip())
    
    a,b = results[i].find("span", class_="subtle loc").text.split(",")
    city.append(a)
    state.append(b[1:3])  

In [179]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser3 = Browser('chrome', **executable_path, headless=True)

for link in url:
    job_desc_url = link 
    browser3.visit(job_desc_url)
    html3 = browser3.html
    soup3 = BeautifulSoup(html3, 'html.parser')
    
    try:
        job_desc = soup3.find("div",class_="jobDescriptionContent desc module pad noMargBot collapsed noPadBot")
        description.append(job_desc.text)
    except:
        print(job_desc_url)
        job_desc = soup3.find("body")
        description.append(f"'{job_desc}'")


https://www.glassdoor.com/partner/jobListing.htm?pos=108&ao=199837&s=58&guid=0000016409de6e828a8eb47df47bb528&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_767e5e46&cb=1529173930079&jobListingId=2802530910


In [180]:
print(f"Len of URL {len(url)}")
print(f"Len of company {len(company)}")
print(f"Len of title {len(title)}")
print(f"Len of state {len(state)}")
print(f"Len of city {len(city)}")
print(f"Len of description {len(description)}")

glassdoor_db = []
for i in range(len(url)):
        glassdoor_dict = {}
        glassdoor_dict["title"] = title[i]
        glassdoor_dict["company"] = company[i]
        glassdoor_dict["city"] = city[i]
        glassdoor_dict["state"] = state[i]
        glassdoor_dict["url"] = url[i]
        glassdoor_dict["description"] = description[i]
        glassdoor_db.append(glassdoor_dict)

Len of URL 30
Len of company 30
Len of title 30
Len of state 30
Len of city 30
Len of description 30


In [181]:
glassdoor_db[0]

{'city': 'Redwood City',
 'company': 'Shutterfly',
 'description': ' The Data Scientist will be responsible for designing and directing experiments and observational studies to optimize our marketing efforts. This role requires an individual with a strong ability to communicate and collaborate across functional teams, in addition to outstanding analytical and critical thinking skills. The role will have a strong focus on experimental design.  Responsibilities:  - Directly impact resource allocation decisions by designing and directing experiments and observational studies  - Partner with the marketing and business strategy teams to define and test hypotheses that answer critical business questions  - Perform deep-dive statistical analysis on large, complex, multi-dimensional datasets.  - Develop and contribute to a base of understanding that allows us to make optimal resource allocation decisions  Qualifications:  - Advanced degree (MS, Ph.D.) in quantitative fields, and 2-3 years of e